In [2]:
import numpy as np
import pandas as pd
import random
import ollama
from dotenv import load_dotenv
import os

POD_ID = os.getenv("POD_ID")

RUNPOD_OLLAMA_URL = f"https://{POD_ID}-11434.proxy.runpod.net"

In [3]:
def query_ollama(prompt: list, model: str = "EEVE-Korean-10.8B") -> str:

    client = ollama.Client(host=RUNPOD_OLLAMA_URL)

    response = client.chat(
        model=model,
        messages=prompt
    )

    return response['message']['content']


In [13]:
def generate_ai_description(prompt):
    messages = [
        {
            'role' : 'system',
            'content' : """ 
            너는 게임 문제를 내는 사람이야.
            다음 단어들 중 일부와 관련있는 Dixit 스타일의 모호하고 추상적인 한 문장을 만들어줘.
            문장의 길이가 너무 길지 않아야 하고, 문장이 짧아야 해.
            단어 예시: 하늘, 망치, 조각, 예술, 새, 나비, 구름, 사다리, 남자

            출력 예시:
            "조각가의 하늘은 아직 다듬어지지 않은 돌이었다."
            """
        },
        {
            'role' : 'user',
            'content': f""" 
            다음 단어들을 보고 일부를 사용해서 떠오르는 모호하고 추상적인 짧은 문장 하나를 만들어줘.
            단어 예시 : {prompt}
            """
        }
    ]

    description = query_ollama(messages)
    
    return description

In [ ]:
import re

def extract_model_answer(model_output: str) -> str:
    """
    모델의 응답 텍스트에서 불필요한 서론/설명을 제거하고,
    '숫자. 단어 집합 내용' 형식의 정답 문장만 추출하여 반환합니다.
    """
    
    # 1. 응답 텍스트를 줄 단위로 분리합니다.
    lines = model_output.strip().split('\n')
    
    # 2. 각 줄을 순회하며 정답 형식(숫자. 으로 시작)을 찾습니다.
    # 정규 표현식: ^\s*(\d+\.)\s*.*$
    #   - ^\s* : 문자열 시작 부분의 공백을 허용
    #   - (\d+\.) : 하나 이상의 숫자와 마침표(예: 0., 1., 2.)를 찾습니다.
    #   - \s*.*$ : 뒤따르는 공백과 모든 문자열을 포함합니다.
    answer_pattern = re.compile(r'^\s*(\d+\.)\s*.*$', re.MULTILINE | re.DOTALL)
    
    for line in lines:
            
        # 정규 표현식 패턴과 일치하는지 확인합니다.
        if answer_pattern.match(line):
            # 패턴에 맞는 첫 번째 줄을 찾았으므로, 해당 줄을 그대로 반환합니다.
            # 이 줄에는 정답 번호와 단어 집합 내용이 모두 포함되어 있습니다.
            return line.strip()
            
    # 정답 형식을 찾지 못했을 경우
    return None


In [61]:
def select_similar_card(user_cards, description):
    cards = list(user_cards)
    print(cards)
    card_list = ""
    for words in cards:
        sent = f"{words[0]}. [{words[1]}]\n"
        card_list += sent

    messages = [
        {
            "role": "system",
            "content": """
            너는 게임 문제를 맞추는 전문 분류기(Classifier)야.
            
            사용자로부터 다음 두 가지 입력을 받을 거야:
            1. '설명 문장': 문제를 풀기 위한 핵심 문장.
            2. '단어 집합 목록': **(파일명, 단어들)** 형식으로 구성된, 정답 후보 목록.
            
            너의 목표는 '설명 문장'에 **가장 의미론적으로 잘 맞는** 단어 집합 하나를 '단어 집합 목록'에서 **선택**하는 거야.
            
            ### 제약 조건 및 출력 형식 ###
            1. **반드시 방금 받은 '단어 집합 목록' 내에서만 선택해야 해.**
            2. **다른 문장, 설명, 주석 없이 오직 선택된 집합의 파일 이름과 그 내용만** 반환해야 해.
            3. **출력의 첫 글자는 무조건 선택된 집합의 파일 이름(예: 1.png)이 되어야 해.**
            4. 출력 형식은 다음과 같아:
            ```
            [선택된 집합의 파일 이름] : [선택된 집합의 단어 내용 전체]
            ```
            
            ### 예시 입력 및 출력 형식 (최종 목표) ###
            
            **사용자 입력 예시:**
            ```
            설명 문장 : 구름은 조각가였고, 하늘은 아직 다듬어지지 않은 돌이었다.
            
            단어 집합 목록 : 
            1.png : 여자, 아이, 바이올린, 음표, ...
            2.png : 여자, 노파, 백발, 앞치마, ...
            3.png : 용, 기사, 칼, 아이, 파랑, ...
            ```

            **너의 출력 (예시):**
            ```
            1.png : 여자, 아이, 바이올린, 음표, 꽃, 나무, 바닥, 오선지, 음악, 옷, 노래
            ```
            """
        },
        {
            'role' : 'user',
            'content' : f""" 
            설명 문장 : {description},
            단어 집합 목록 : {card_list}
            """
        }
    ]

    select_response = query_ollama(messages)

    select_response = extract_model_answer(select_response)

    print(select_response)
    file_num = select_response[0]

    
    if select_response is not None:
        for i, card in enumerate(cards):
            if card[0][0] == file_num:
                selected_card = cards[i]
    else:
        print("random select")
        selected_card = random.choice(cards)

    return selected_card, user_cards

In [6]:
card = ("1.png", "여자, 아이, 바이올린, 음표, 꽃, 나무, 바닥, 오선지, 음악, 옷, 노래")

In [15]:
description = generate_ai_description(card)

In [66]:
print(description)

words = {("1.png", "여자, 아이, 바이올린, 음표, 꽃, 나무, 바닥, 오선지, 음악, 옷, 노래"),
         ("2.png", "여자, 노파, 백발, 앞치마, 꽃, 아이, 머리, 빨강, 화분"),
         ("3.png", "용, 기사, 칼 , 아이, 파랑, 싸움, 날개"),
        }

select, _ = select_similar_card(words, description)
print(select)

"음악의 천사들은 하늘에서 부드럽게 내려와 그녀의 영혼을 매혹할 노래를 연주한다."
[('3.png', '용, 기사, 칼 , 아이, 파랑, 싸움, 날개'), ('2.png', '여자, 노파, 백발, 앞치마, 꽃, 아이, 머리, 빨강, 화분'), ('1.png', '여자, 아이, 바이올린, 음표, 꽃, 나무, 바닥, 오선지, 음악, 옷, 노래')]
1.png : 여자, 아이, 바이올린, 음표, 꽃, 나무, 바닥, 오선지, 음악, 옷, 노래
('1.png', '여자, 아이, 바이올린, 음표, 꽃, 나무, 바닥, 오선지, 음악, 옷, 노래')


In [68]:
print(select)

('1.png', '여자, 아이, 바이올린, 음표, 꽃, 나무, 바닥, 오선지, 음악, 옷, 노래')
